## Data Loading and Initial Exploration

This section loads the dataset and performs initial inspections to understand its structure and content.

In [656]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
warnings.filterwarnings('ignore')

In [657]:
df = pd.read_excel('/content/Copy of Analytics Position Case Study.xlsx')

In [658]:
gameplay_df = pd.read_excel('/content/Copy of Analytics Position Case Study.xlsx', sheet_name="User Gameplay data", skiprows=3, header=0)

In [659]:
deposit_df = pd.read_excel('/content/Copy of Analytics Position Case Study.xlsx', sheet_name="Deposit Data" , skiprows=3, header=0)

In [660]:
withdrawal_df = pd.read_excel('/content/Copy of Analytics Position Case Study.xlsx', sheet_name="Withdrawal Data" , skiprows=3, header=0)

In [661]:
gameplay_df.head(10)

,User ID,Games Played,Datetime
0,851,1,2022-01-10 00:00:00
1,717,1,2022-01-10 00:00:00
2,456,1,2022-01-10 00:00:00
3,424,1,2022-01-10 00:00:00
4,845,1,2022-01-10 00:00:00
5,15,1,2022-01-10 00:00:00
6,738,1,2022-01-10 00:00:00
7,748,1,2022-01-10 00:01:00
8,582,1,2022-01-10 00:01:00
9,39,1,2022-01-10 00:01:00


In [662]:
deposit_df.head(10)

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500
5,39,2022-01-10 00:11:00,4000
6,539,2022-01-10 00:13:00,500
7,560,2022-01-10 00:15:00,2500
8,30,2022-01-10 00:16:00,25000
9,367,2022-01-10 00:18:00,1000


In [663]:
withdrawal_df.head(10)

,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473
5,47,2022-01-10 01:01:00,3637
6,302,2022-01-10 01:11:00,2000
7,461,2022-01-10 01:17:00,9800
8,12,2022-01-10 01:27:00,1800
9,530,2022-01-10 01:32:00,16000


In [664]:
gameplay_df.info() , deposit_df.info() , withdrawal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355266 entries, 0 to 355265
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   User ID       355266 non-null  int64 
 1   Games Played  355266 non-null  int64 
 2   Datetime      355266 non-null  object
dtypes: int64(2), object(1)
memory usage: 8.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17438 entries, 0 to 17437
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User Id   17438 non-null  int64 
 1   Datetime  17438 non-null  object
 2   Amount    17438 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 408.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3566 entries, 0 to 3565
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User Id   3566 non-null   int64 
 1   Datetime  3566 non-null   object
 2   Amount    35

(None, None, None)

In [665]:
gameplay_df.isnull().sum() , deposit_df.isnull().sum() , withdrawal_df.isnull().sum()

(User ID         0
 Games Played    0
 Datetime        0
 dtype: int64,
 User Id     0
 Datetime    0
 Amount      0
 dtype: int64,
 User Id     0
 Datetime    0
 Amount      0
 dtype: int64)

## **Aggregating the deposit and withdrawal data by 'User ID' first, we get a summary of the total activity for each user.**

In [666]:
# Grouping data for aggregation
deposit_df.groupby('User Id')['Amount'].sum()

,Amount
User Id,
1,5000
2,567000
3,40000
4,1750
5,74100
...,...
994,1400
995,34500
996,96400


In [667]:
deposit = deposit_df.groupby('User Id')['Amount'].sum().reset_index()
withdrawal = withdrawal_df.groupby('User Id')['Amount'].sum().reset_index() # Changed 'User ID' to 'User Id' here

In [668]:
# Grouping data for aggregation
deposit.rename(columns={'Amount': 'TotalDeposit'}, inplace=True)
withdrawal.rename(columns={'Amount': 'TotalWithdrawal'}, inplace=True)


In [669]:
# Merging datasets on common keys
merged_df = gameplay_df.merge(deposit, left_on='User ID', right_on='User Id', how='left')
merged_df = merged_df.merge(withdrawal,left_on='User ID', right_on='User Id', how='left')

In [670]:
merged_df.columns

Index(['User ID', 'Games Played', 'Datetime', 'User Id_x', 'TotalDeposit',
       'User Id_y', 'TotalWithdrawal'],
      dtype='object')

In [671]:
# Merging datasets on common keys
merged_df.drop('User Id_x', axis=1, inplace=True)
merged_df.drop('User Id_y', axis=1, inplace=True)


In [672]:
merged_df['User ID'].isnull().sum() # looking for null

np.int64(0)

In [673]:
merged_df.isnull().sum()

,0
User ID,0
Games Played,0
Datetime,0
TotalDeposit,63062
TotalWithdrawal,92148


In [674]:
merged_df.fillna(0, inplace=True) # fill nan values with 0

In [675]:
merged_df.head()

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal
0,851,1,2022-01-10 00:00:00,55800.0,29900.0
1,717,1,2022-01-10 00:00:00,117400.0,42000.0
2,456,1,2022-01-10 00:00:00,111000.0,65000.0
3,424,1,2022-01-10 00:00:00,115392.0,0.0
4,845,1,2022-01-10 00:00:00,8000.0,358310.0


In [676]:
merged_df['Games Played'].isnull().sum()

np.int64(0)

In [677]:
merged_df.fillna(0, inplace=True)

In [678]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355266 entries, 0 to 355265
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   User ID          355266 non-null  int64  
 1   Games Played     355266 non-null  int64  
 2   Datetime         355266 non-null  object 
 3   TotalDeposit     355266 non-null  float64
 4   TotalWithdrawal  355266 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 13.6+ MB


In [679]:
merged_df[['Games Played','TotalDeposit' , 'TotalWithdrawal']] = merged_df[['Games Played','TotalDeposit' , 'TotalWithdrawal']].astype(int)

In [680]:
merged_df.head()

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal
0,851,1,2022-01-10 00:00:00,55800,29900
1,717,1,2022-01-10 00:00:00,117400,42000
2,456,1,2022-01-10 00:00:00,111000,65000
3,424,1,2022-01-10 00:00:00,115392,0
4,845,1,2022-01-10 00:00:00,8000,358310


In [681]:
merged_df.isnull().sum()

,0
User ID,0
Games Played,0
Datetime,0
TotalDeposit,0
TotalWithdrawal,0


In [682]:
#create no of deposits for loyalty calculation
merged_df["Number of Deposits"] = merged_df["User ID"].map(deposit_df["User Id"].value_counts())


In [683]:
merged_df["Number of Withdrawals"] = merged_df["User ID"].map(withdrawal_df["User Id"].value_counts())


In [684]:
merged_df[["Number of Deposits", "Number of Withdrawals"]] = merged_df[["Number of Deposits", "Number of Withdrawals"]].fillna(0)


In [685]:
merged_df.isnull().sum()

,0
User ID,0
Games Played,0
Datetime,0
TotalDeposit,0
TotalWithdrawal,0
Number of Deposits,0
Number of Withdrawals,0


## **Calculate Loyalty points per user**

In [686]:
merged_df['Loyalty Points'] = merged_df['TotalDeposit'] * 0.01 + merged_df['TotalWithdrawal'] * 0.005 + merged_df['Games Played'] * 0.2 + (merged_df['Number of Deposits']-merged_df['Number of Withdrawals'])*0.001

In [687]:
merged_df.head()

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points
0,851,1,2022-01-10 00:00:00,55800,29900,45.0,2.0,707.743
1,717,1,2022-01-10 00:00:00,117400,42000,52.0,2.0,1384.250
2,456,1,2022-01-10 00:00:00,111000,65000,4.0,1.0,1435.203
3,424,1,2022-01-10 00:00:00,115392,0,57.0,0.0,1154.177
4,845,1,2022-01-10 00:00:00,8000,358310,2.0,14.0,1871.738


In [688]:


# Step 1: Convert Datetime column
merged_df["Datetime"] = pd.to_datetime(merged_df["Datetime"])

# Step 2: Add Slot column
merged_df["Slot"] = merged_df["Datetime"].dt.hour.apply(lambda x: "S1" if x < 12 else "S2")

merged_df["DateOnly"] = merged_df["Datetime"].dt.date



## **Find Playerwise Loyalty points earned by Players in the following slots:**

 ## 2 nd october slot 1

In [689]:
slot_2oct_s1 = merged_df[(merged_df["DateOnly"] == datetime.date(2022, 10, 2)) & (merged_df["Slot"] == "S1")]


In [690]:
slot_2oct_s1

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points,Slot,DateOnly


## 16th October Slot 2

In [691]:
slot_16oct_S2 =merged_df[(merged_df["DateOnly"] == datetime.date(2022,10,16)) & (merged_df["Slot"] == "S2")]

In [692]:
slot_16oct_S2

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points,Slot,DateOnly
177649,442,1,2022-10-16 12:00:00,40000,46308,2.0,2.0,631.740,S2,2022-10-16
177650,460,1,2022-10-16 12:00:00,0,205000,0.0,2.0,1025.198,S2,2022-10-16
177651,524,1,2022-10-16 12:00:00,0,0,0.0,0.0,0.200,S2,2022-10-16
177652,663,1,2022-10-16 12:00:00,425000,290200,52.0,35.0,5701.217,S2,2022-10-16
177653,843,1,2022-10-16 12:00:00,10000,104500,1.0,9.0,622.692,S2,2022-10-16
...,...,...,...,...,...,...,...,...,...,...
183280,391,1,2022-10-16 23:59:00,0,0,0.0,0.0,0.200,S2,2022-10-16
183281,620,1,2022-10-16 23:59:00,15000,6000,3.0,1.0,180.202,S2,2022-10-16
183282,9,1,2022-10-16 23:59:00,193684,171456,109.0,9.0,2794.420,S2,2022-10-16
183283,39,1,2022-10-16 23:59:00,134500,151320,44.0,8.0,2101.836,S2,2022-10-16


## 18th October Slot 1

In [693]:
slot_18oct_s1 = merged_df[(merged_df["DateOnly"] == datetime.date(2022, 10, 18)) & (merged_df["Slot"] == "S1")]


In [694]:
slot_18oct_s1

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points,Slot,DateOnly
194770,754,1,2022-10-18 00:00:00,207500,158059,24.0,22.0,2865.497,S1,2022-10-18
194771,449,1,2022-10-18 00:00:00,198730,106153,13.0,4.0,2518.274,S1,2022-10-18
194772,748,1,2022-10-18 00:00:00,74500,75000,13.0,3.0,1120.210,S1,2022-10-18
194773,137,1,2022-10-18 00:00:00,103000,228000,13.0,16.0,2170.197,S1,2022-10-18
194774,956,1,2022-10-18 00:01:00,1200,0,1.0,0.0,12.201,S1,2022-10-18
...,...,...,...,...,...,...,...,...,...,...
200578,828,1,2022-10-18 11:59:00,140500,59495,31.0,7.0,1702.699,S1,2022-10-18
200579,582,1,2022-10-18 11:59:00,0,0,0.0,0.0,0.200,S1,2022-10-18
200580,218,1,2022-10-18 11:59:00,113750,25000,55.0,1.0,1262.754,S1,2022-10-18
200581,774,1,2022-10-18 11:59:00,110000,331000,2.0,6.0,2755.196,S1,2022-10-18


## 26th October Slot S2

In [695]:
slot_26oct_s2 = merged_df[(merged_df["DateOnly"] == datetime.date(2022, 10, 26)) & (merged_df["Slot"] == "S2")]


In [696]:
slot_26oct_s2

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points,Slot,DateOnly
292333,659,1,2022-10-26 12:00:00,38000,41800,6.0,6.0,589.200,S2,2022-10-26
292334,765,1,2022-10-26 12:00:00,0,31000,0.0,3.0,155.197,S2,2022-10-26
292335,716,1,2022-10-26 12:00:00,4433,7000,12.0,2.0,79.540,S2,2022-10-26
292336,605,1,2022-10-26 12:00:00,70050,0,49.0,0.0,700.749,S2,2022-10-26
292337,270,1,2022-10-26 12:01:00,56800,30679,15.0,2.0,721.608,S2,2022-10-26
...,...,...,...,...,...,...,...,...,...,...
298177,379,1,2022-10-26 23:59:00,29448,0,75.0,0.0,294.755,S2,2022-10-26
298178,237,1,2022-10-26 23:59:00,70000,70000,3.0,1.0,1050.202,S2,2022-10-26
298179,27,1,2022-10-26 23:59:00,133700,179581,24.0,2.0,2235.127,S2,2022-10-26
298180,168,1,2022-10-26 23:59:00,330700,96400,76.0,11.0,3789.265,S2,2022-10-26


 ## **Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October.**


In [697]:
october_df = merged_df[(merged_df["Datetime"].dt.month == 10)]


In [698]:
october_df

,User ID,Games Played,Datetime,TotalDeposit,TotalWithdrawal,Number of Deposits,Number of Withdrawals,Loyalty Points,Slot,DateOnly
103487,565,1,2022-10-10 00:00:00,159510,317936,70.0,34.0,3185.016,S1,2022-10-10
103488,989,1,2022-10-10 00:00:00,240000,1339000,12.0,20.0,9095.192,S1,2022-10-10
103489,887,1,2022-10-10 00:00:00,0,0,0.0,0.0,0.200,S1,2022-10-10
103490,618,1,2022-10-10 00:00:00,280721,58500,143.0,5.0,3100.048,S1,2022-10-10
103491,460,1,2022-10-10 00:01:00,0,205000,0.0,2.0,1025.198,S1,2022-10-10
...,...,...,...,...,...,...,...,...,...,...
355261,658,1,2022-10-31 23:59:00,142065,0,40.0,0.0,1420.890,S2,2022-10-31
355262,582,1,2022-10-31 23:59:00,0,0,0.0,0.0,0.200,S2,2022-10-31
355263,272,1,2022-10-31 23:59:00,0,189202,0.0,8.0,946.202,S2,2022-10-31
355264,563,1,2022-10-31 23:59:00,15900,0,31.0,0.0,159.231,S2,2022-10-31


In [699]:
october_df["DateOnly"] = october_df["Datetime"].dt.date


In [700]:
# Grouping data for aggregation
oct_summary = october_df.groupby(["User ID", "DateOnly"]).agg({
    "TotalDeposit": "sum",
    "TotalWithdrawal": "sum",
    "Games Played": "sum",
    "Number of Deposits": "sum",
    "Number of Withdrawals": "sum",
    "Loyalty Points": "sum"
}).reset_index()

In [701]:
oct_summary = oct_summary.sort_values(by=["Loyalty Points", "Games Played"],ascending=[False, False]).reset_index(drop=True)

In [702]:
oct_summary

,User ID,DateOnly,TotalDeposit,TotalWithdrawal,Games Played,Number of Deposits,Number of Withdrawals,Loyalty Points
0,365,2022-10-20,43540000,252326900,140,1400.0,4620.0,1697059.280
1,365,2022-10-31,42296000,245117560,136,1360.0,4488.0,1648571.872
2,365,2022-10-10,41985000,243315225,135,1350.0,4455.0,1636450.020
3,365,2022-10-18,41363000,239710555,133,1330.0,4389.0,1612206.316
4,365,2022-10-21,41052000,237908220,132,1320.0,4356.0,1600084.464
...,...,...,...,...,...,...,...,...
13124,988,2022-10-28,0,0,1,0.0,0.0,0.200
13125,988,2022-10-29,0,0,1,0.0,0.0,0.200
13126,997,2022-10-17,0,0,1,0.0,0.0,0.200
13127,997,2022-10-30,0,0,1,0.0,0.0,0.200


In [703]:
oct_summary["Rank"] = range(1, len(oct_summary) + 1)

In [704]:
oct_summary

,User ID,DateOnly,TotalDeposit,TotalWithdrawal,Games Played,Number of Deposits,Number of Withdrawals,Loyalty Points,Rank
0,365,2022-10-20,43540000,252326900,140,1400.0,4620.0,1697059.280,1
1,365,2022-10-31,42296000,245117560,136,1360.0,4488.0,1648571.872,2
2,365,2022-10-10,41985000,243315225,135,1350.0,4455.0,1636450.020,3
3,365,2022-10-18,41363000,239710555,133,1330.0,4389.0,1612206.316,4
4,365,2022-10-21,41052000,237908220,132,1320.0,4356.0,1600084.464,5
...,...,...,...,...,...,...,...,...,...
13124,988,2022-10-28,0,0,1,0.0,0.0,0.200,13125
13125,988,2022-10-29,0,0,1,0.0,0.0,0.200,13126
13126,997,2022-10-17,0,0,1,0.0,0.0,0.200,13127
13127,997,2022-10-30,0,0,1,0.0,0.0,0.200,13128


## **What is the average deposit amount?**

In [705]:
avg_deposit_amount = deposit_df["Amount"].mean()
print(avg_deposit_amount)


5492.185399701801


## **What is the average deposit amount per user in a month?**

In [706]:
monthly_user_deposit = deposit_df.groupby("User Id")["Amount"].sum()
avg_deposit_per_user = monthly_user_deposit.mean()

In [707]:
print(avg_deposit_per_user)


104669.64918032786


## **What is the average number of games played per user?"**

In [708]:
monthly_games = gameplay_df.groupby("User ID")["Games Played"].sum()
avg_games_per_user = monthly_games.mean()

In [709]:
print(avg_games_per_user)

355.267


## **Part B - How much bonus should be allocated to leaderboard players?**

In [710]:
top_50 = oct_summary.head(50)

In [711]:
top_50

,User ID,DateOnly,TotalDeposit,TotalWithdrawal,Games Played,Number of Deposits,Number of Withdrawals,Loyalty Points,Rank
0,365,2022-10-20,43540000,252326900,140,1400.0,4620.0,1697059.280,1
1,365,2022-10-31,42296000,245117560,136,1360.0,4488.0,1648571.872,2
2,365,2022-10-10,41985000,243315225,135,1350.0,4455.0,1636450.020,3
3,365,2022-10-18,41363000,239710555,133,1330.0,4389.0,1612206.316,4
4,365,2022-10-21,41052000,237908220,132,1320.0,4356.0,1600084.464,5
5,365,2022-10-16,39497000,228896545,127,1270.0,4191.0,1539475.204,6
6,365,2022-10-26,38875000,225291875,125,1250.0,4125.0,1515231.500,7
7,365,2022-10-14,36387000,210873195,117,1170.0,3861.0,1418256.684,8
8,365,2022-10-15,36387000,210873195,117,1170.0,3861.0,1418256.684,9
9,365,2022-10-19,36387000,210873195,117,1170.0,3861.0,1418256.684,10


In [712]:
def bonus_multiplier(rank):
    if rank == 1:
        return 3.0
    elif rank == 2:
        return 2.5
    elif rank == 3:
        return 2.2
    elif rank in [4, 5]:
        return 2.0
    elif 6 <= rank <= 10:
        return 1.8
    elif 11 <= rank <= 30:
        return 1.5
    else:
        return 1.0

top_50["Multiplier"] = top_50["Rank"].apply(bonus_multiplier)
top_50["Weighted Score"] = top_50["Loyalty Points"] * top_50["Multiplier"]

# Final bonus distribution
top_50["Bonus ₹"] = top_50["Weighted Score"] / top_50["Weighted Score"].sum() * 50_000


In [713]:
top_50

,User ID,DateOnly,TotalDeposit,TotalWithdrawal,Games Played,Number of Deposits,Number of Withdrawals,Loyalty Points,Rank,Multiplier,Weighted Score,Bonus ₹
0,365,2022-10-20,43540000,252326900,140,1400.0,4620.0,1697059.280,1,3.0,5.091178e+06,2713.923600
1,365,2022-10-31,42296000,245117560,136,1360.0,4488.0,1648571.872,2,2.5,4.121430e+06,2196.985771
2,365,2022-10-10,41985000,243315225,135,1350.0,4455.0,1636450.020,3,2.2,3.600190e+06,1919.131688
3,365,2022-10-18,41363000,239710555,133,1330.0,4389.0,1612206.316,4,2.0,3.224413e+06,1718.818280
4,365,2022-10-21,41052000,237908220,132,1320.0,4356.0,1600084.464,5,2.0,3.200169e+06,1705.894834
5,365,2022-10-16,39497000,228896545,127,1270.0,4191.0,1539475.204,6,1.8,2.771055e+06,1477.149845
6,365,2022-10-26,38875000,225291875,125,1250.0,4125.0,1515231.500,7,1.8,2.727417e+06,1453.887643
7,365,2022-10-14,36387000,210873195,117,1170.0,3861.0,1418256.684,8,1.8,2.552862e+06,1360.838834
8,365,2022-10-15,36387000,210873195,117,1170.0,3861.0,1418256.684,9,1.8,2.552862e+06,1360.838834
9,365,2022-10-19,36387000,210873195,117,1170.0,3861.0,1418256.684,10,1.8,2.552862e+06,1360.838834


## **Part C -Would you say the loyalty point formula is fair or unfair?**

**Original loyalty points formula**
* merged_df['Loyalty Points'] = merged_df['TotalDeposit'] * 0.01 + merged_df['TotalWithdrawal'] * 0.005 + merged_df['Games Played'] * 0.2 + (merged_df['Number of Deposits']-merged_df['Number of Withdrawals'])*0.00

**The original loyalty formula, while functional, was fundamentally skewed in favor of users with high financial capacity. By heavily weighting total deposits and even including withdrawals as a positive factor, the system unintentionally rewarded users who could simply deposit and withdraw large amounts — without necessarily being active or loyal to the platform. The impact of gameplay and engagement was significantly diluted, and there was no incentive for users to remain consistently active over time. As a result, the formula favored short-term, high-value users over consistent, engaged players.**

**Revised loyalty points formula**
* ['Loyalty Points'] = ['TotalDeposit']*0.01) +(['Games Played'] * 0.2) + (['Consecutive Active Days'] * 0.1) +( ['Number of Deposits']-['Number of Withdrawals'])*0.001

**The revised formula addresses these flaws by eliminating withdrawal-based rewards entirely and introducing a new component: Consecutive Active Days. This addition ensures that players who consistently log in and participate are fairly rewarded, even if they are not the biggest depositors. Gameplay remains a core factor in loyalty calculation, and deposits are still rewarded — but within a structure that also values time, activity, and real user commitment. The formula also retains a subtle nod to transaction frequency through the deposit-minus-withdrawal term, but without allowing it to override actual engagement.**

## **Conclusion**

**Loyalty Points System Evaluation and Redesign Report**

**Submitted by:** Ridham
**Context:** Analytics Case Study for Real-Money Gaming Platform
**Goal:** Evaluate and improve the current loyalty point system to ensure fairness, engagement, and alignment with business goals.

---

### **1. Introduction**

This report presents a detailed evaluation and redesign of the loyalty points system used in a real-money gaming platform. The platform rewards users based on their activity and transaction behavior. The assignment required a data-driven analysis of the existing loyalty point formula, followed by a proposal for a fairer and more effective version. It also included the challenge of designing a suitable bonus distribution method for the top 50 players.

---

### **2. Existing Loyalty Points Formula**

The current loyalty point calculation is defined as:

```
Loyalty Points =
    0.01 × TotalDeposit +
    0.005 × TotalWithdrawal +
    0.2 × GamesPlayed +
    0.001 × (Number of Deposits − Number of Withdrawals)
```

**Observation:** This formula heavily favors users with high deposits and even rewards withdrawals, which can negatively affect the platform’s financial health. It undervalues gameplay and completely ignores user consistency or streaks.

---

### **3. Identified Issues and Difficulties Faced**

During the analysis and implementation, the following issues and challenges were encountered:

* **Unfair Advantage to High Depositors:** Users who deposited large amounts but barely played were still topping the leaderboard.
* **Withdrawals being rewarded:** Giving points for withdrawing money contradicted business interests.
* **Missing Consistency Measurement:** No part of the formula measured loyalty in terms of daily activity or sustained engagement.
* **Overwriting gameplay data:** While merging dataframes, the "Games Played" column was often overwritten or duplicated, leading to 0 values.
* **Bias Detection:** It was difficult to see bias until bonus distribution comparisons were done using Excel.
* **Crash during merge:** Filling NaN after merging deposit/withdrawal data led to data corruption or loss.
* **Manual creation of time slots (S1/S2):** Required transforming datetime into daily time segments manually.
* **Top 50 player bonus fairness:** Needed to design a scalable and fair bonus system that rewards effort, not just money.

---

### **4. Revised Loyalty Formula**

To address the limitations of the original approach, a new loyalty formula is proposed:

```
Loyalty Points =
    0.01 × TotalDeposit +
    0.2 × GamesPlayed +
    0.1 × Consecutive Active Days +
    0.001 × (Number of Deposits − Number of Withdrawals)
```

**Explanation of Components:**

* **TotalDeposit:** Still valued, but balanced with other metrics.
* **GamesPlayed:** Weight increased to properly reward active users.
* **Consecutive Active Days:** New addition to reward consistency and daily commitment.
* **Removed Withdrawal Points:** Withdrawals no longer give positive loyalty points.

---

### **5. Bonus Pool Distribution Strategy**

Instead of a flat or tiered manual bonus model, a **weighted proportional model** was used. Each top 50 player receives a bonus based on their loyalty points after applying the new formula.

This ensures:

* Top performers are fairly rewarded.
* Users with no deposit or minimal gameplay are filtered out.
* Bonus distribution aligns with platform revenue and engagement.

---

### **6. Outcome and Impact**

After applying the new formula and running comparison datasets:

* **Bias dropped significantly.** Players who previously benefited unfairly from withdrawals no longer appeared in the top 10.
* **Engaged players rose in rank.** Users with high game counts and consistent logins moved up.
* **Withdrawals stopped influencing rankings.**

A detailed Excel file was created showing bonus distributions from all three formulas:

* Original formula
* Previous suggested formula (with caps)
* Final balanced formula (with streaks)

Top 10 lists were compared and revealed **completely different players** ranking high across each formula, proving that formula design has massive impact.

---

### **7. Conclusion**

The redesigned loyalty formula solves key problems in the original model and creates a sustainable, scalable rewards system. It fairly acknowledges deposit contributions, encourages real gameplay, and rewards ongoing engagement through daily activity.

This formula aligns well with the business model of a real-money gaming platform. It filters out freeloaders, discourages manipulative deposit-withdraw cycles, and promotes long-term retention. In conclusion, this version of the formula provides a fair, profitable, and engagement-oriented loyalty system.
